In [1]:
import tensorflow as tf
import tools.datasets.mnist3 as mnist_tools
import tools.datasets.mnistm as mnistm_tools 
import tools.semisup as semisup
import numpy as np
import architectures as arch
from functools import partial

import tools.visualization as vis
import tools.updated_semisup as up

%load_ext autoreload
%autoreload 2

import sys

mnist_train_images, mnist_train_labels = mnist_tools.get_data('train')
mnist_test_images, mnist_test_labels = mnist_tools.get_data('test')
mnistm_train_images, mnistm_train_labels = mnistm_tools.get_data('train')
mnistm_test_images, mnistm_test_labels = mnistm_tools.get_data('test')

/home/frankzl/.envs/env36-ml/.venv/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


Extracting /home/frankzl/datasets/mnist//train-images-idx3-ubyte.gz
Extracting /home/frankzl/datasets/mnist//train-labels-idx1-ubyte.gz
Extracting /home/frankzl/datasets/mnist//t10k-images-idx3-ubyte.gz
Extracting /home/frankzl/datasets/mnist//t10k-labels-idx1-ubyte.gz


In [2]:
# labeled samples used per class
# sup_per_class = 10
sup_per_class = 1280
sup_seed = -1
# labeled samples per class per batch
sup_per_batch = 100
# unlabeled samples per batch
unsup_batch_size = 1000
unsup_samples = -1

learning_rate = 1e-4
decay_steps = 9000
decay_factor = 0.33
logit_weight = 1.0

max_steps = 2000
eval_interval = 500

log_dir = "logs/training/mnist-mnistmK/model"

seed = 1

IMAGE_SHAPE = mnist_tools.IMAGE_SHAPE
NUM_LABELS = mnist_tools.NUM_LABELS

# [10 (classes), 10 (samples), 28, 28, 1]
sup_by_label = semisup.sample_by_label(mnist_train_images, mnist_train_labels,
                        sup_per_class, NUM_LABELS, seed)

visit_weight_envelope = "linear"
visit_weight = 2
visit_weight_envelope_steps = 1
visit_weight_envelope_delay = 500

walker_weight_envelope = "linear"
walker_weight = 10
walker_weight_envelope_steps = 1
walker_weight_envelope_delay = 500

TARGET_SHAPE = mnistm_tools.IMAGE_SHAPE
TEST_SHAPE   = TARGET_SHAPE

image_shape = IMAGE_SHAPE
new_shape   = TARGET_SHAPE
emb_size    = 128

sampled_unsup_images = mnistm_train_images
sampled_unsup_labels = mnistm_train_labels

In [3]:
store_dir = "logs/training/mnist-mnistK-s0.5/model"

In [4]:
from ipywidgets import IntProgress, Layout
from train import apply_envelope

graph = tf.Graph()

with graph.as_default():
    
    unsup_data = up.create_input(sampled_unsup_images, sampled_unsup_labels, unsup_batch_size)
    unsup_it   = unsup_data.make_initializable_iterator()
    
    t_unsup_images,t_unsup_labels = unsup_it.get_next()
    
    sup_data,sup_label = up.create_per_class_inputs_v2(sup_by_label, sup_per_batch)
    sup_it             = sup_data.repeat().make_one_shot_iterator()
    sup_it_label       = sup_label.repeat().make_one_shot_iterator()
    
    # Apply augmentation
    def _random_invert(inputs, _):
        randu = tf.random_uniform(
            shape=[sup_per_batch * NUM_LABELS], minval=0.,
            maxval=1.,
            dtype=tf.float32)
        randu = tf.cast(tf.less(randu, 0.5), tf.float32)
        randu = tf.expand_dims(randu, 1)
        randu = tf.expand_dims(randu, 1)
        randu = tf.expand_dims(randu, 1)
        inputs = tf.cast(inputs, tf.float32)
        return tf.abs(inputs - 255 * randu)

    augmentation_function = _random_invert
    
    model_func = partial(
        arch.svhn_model,
        new_shape=new_shape,
        augmentation_function=augmentation_function,
        img_shape=image_shape,
        emb_size=emb_size
    )
    
    model = semisup.SemisupModel(model_func, NUM_LABELS, IMAGE_SHAPE,
                                 test_in=tf.placeholder(np.float32, [None] + TEST_SHAPE, 'test_in')
                                )
    
    t_sup_images, t_sup_labels = tf.concat( sup_it.get_next(), 0), tf.concat( sup_it_label.get_next(), 0)
    
    t_sup_emb   = model.image_to_embedding(t_sup_images)
    t_unsup_emb = model.image_to_embedding(t_unsup_images)
    
    t_sup_logit = model.embedding_to_logit(t_sup_emb)
    
    visit_weight = apply_envelope(
        type = visit_weight_envelope,
        step = model.step,
        final_weight = visit_weight,
        growing_steps = visit_weight_envelope_steps,
        delay = visit_weight_envelope_delay
    )
    
    walker_weight = apply_envelope(
        type = walker_weight_envelope,
        step = model.step,
        final_weight = walker_weight,
        growing_steps = walker_weight_envelope_steps,
        delay = walker_weight_envelope_delay
    )
    
    tf.summary.scalar("Weights_Visit", visit_weight)
    tf.summary.scalar("Weight_Walker", walker_weight)
    
    model.add_logit_loss(t_sup_logit, t_sup_labels, weight=logit_weight)
    
    #model.add_semisup_loss(t_sup_emb, t_unsup_emb, t_sup_labels, visit_weight=visit_weight, walker_weight=walker_weight)
    equality_matrix = tf.equal(tf.reshape(t_sup_labels, [-1, 1]), t_sup_labels)
    equality_matrix = tf.cast(equality_matrix, tf.float32)
    p_target = (equality_matrix / tf.reduce_sum(
        equality_matrix, [1], keepdims=True))

    embedding_dim = t_sup_emb.shape[1]
    reshaped_semb = tf.reshape( t_sup_emb, [-1, 1, embedding_dim] )
    reshaped_uemb = tf.reshape( t_unsup_emb, [-1, 1, embedding_dim] )
    
    stacked_semb = tf.stack(unsup_batch_size*[t_sup_emb], 1)
    stacked_uemb = tf.stack(sup_per_batch*NUM_LABELS*[t_unsup_emb], 1)
    
    uemb_T = tf.transpose(stacked_uemb, perm=[1,0,2])
     
    sigma = 0.5
    pairwise_dist = (stacked_semb - uemb_T)#, axis=2)
    pairwise_norm = tf.norm( pairwise_dist, axis=2)
    pairwise_sq   = tf.square(pairwise_norm)
    
    match_ab   = tf.exp(- tf.divide( pairwise_sq, tf.constant(2*sigma**2, dtype=tf.float32)), name='match_ab')
    
    p_ab = tf.nn.softmax(match_ab, name='p_ab')
    p_ba = tf.nn.softmax(tf.transpose(match_ab), name='p_ba')
    p_aba = tf.matmul(p_ab, p_ba, name='p_aba')

    model.create_walk_statistics(p_aba, equality_matrix)
    
    loss_aba = tf.losses.softmax_cross_entropy(
        p_target,
        tf.log(1e-8 + p_aba),
        weights=walker_weight,
        scope='loss_aba')
    
    mab_dt, pab_dt, paba_dt, semb_dt, uemb_dt = tf.gradients([loss_aba], [match_ab, p_ab, p_aba, t_sup_emb, t_unsup_emb])
    
    model.add_visit_loss(p_ab, visit_weight)

    tf.summary.scalar('Loss_aba', loss_aba)
    
    t_learning_rate = tf.train.exponential_decay(
        learning_rate,
        model.step,
        decay_steps,
        decay_factor,
        staircase = True
    )
    
    train_op = model.create_train_op(t_learning_rate)
    summary_op = tf.summary.merge_all()
    
    summary_writer = tf.summary.FileWriter(store_dir, graph)
    
    saver = tf.train.Saver(max_to_keep=30)

---------
Variables: name (type shape) [size]
---------
net/conv1/weights:0 (float32_ref 3x3x3x32) [864, bytes: 3456]
net/conv1/biases:0 (float32_ref 32) [32, bytes: 128]
net/conv1_2/weights:0 (float32_ref 3x3x32x32) [9216, bytes: 36864]
net/conv1_2/biases:0 (float32_ref 32) [32, bytes: 128]
net/conv1_3/weights:0 (float32_ref 3x3x32x32) [9216, bytes: 36864]
net/conv1_3/biases:0 (float32_ref 32) [32, bytes: 128]
net/conv2_1/weights:0 (float32_ref 3x3x32x64) [18432, bytes: 73728]
net/conv2_1/biases:0 (float32_ref 64) [64, bytes: 256]
net/conv2_2/weights:0 (float32_ref 3x3x64x64) [36864, bytes: 147456]
net/conv2_2/biases:0 (float32_ref 64) [64, bytes: 256]
net/conv2_3/weights:0 (float32_ref 3x3x64x64) [36864, bytes: 147456]
net/conv2_3/biases:0 (float32_ref 64) [64, bytes: 256]
net/conv3/weights:0 (float32_ref 3x3x64x128) [73728, bytes: 294912]
net/conv3/biases:0 (float32_ref 128) [128, bytes: 512]
net/conv3_2/weights:0 (float32_ref 3x3x128x128) [147456, bytes: 589824]
net/conv3_2/biases:

In [5]:
import time
import datetime
log_dir

'logs/training/mnist-mnistmK/model'

In [6]:
test_images = mnistm_test_images[:2000]
test_labels = mnistm_test_labels[:2000]


from IPython.display import display

max_steps = 1002
f = IntProgress(min=0, max=max_steps, layout= Layout(width="100%")) # instantiate the bar
display(f) # display the bar

sesh = tf.Session(graph = graph, config=tf.ConfigProto(intra_op_parallelism_threads=2, allow_soft_placement=True))

eval_interval = 100

p_aba_list = []
match_ab_list = []
t_sup_emb_list = []
t_unsup_emb_list = []

model_checkpoint = f"{log_dir}-{501}"

with sesh as sess:
    
    saver = tf.train.Saver(max_to_keep=30)
    saver.restore(sess, model_checkpoint)
    
    sess.run(unsup_it.initializer)
    
    epoch = 0
    
    for step in range(501, max_steps):
        t0 = time.time()
        try:
            loss, _, summaries = sess.run([model.train_loss, train_op, summary_op])
        except tf.errors.OutOfRangeError:
            sess.run(unsup_it.initializer)
            epoch += 1
            
            loss, _, summaries = sess.run([model.train_loss, train_op, summary_op])
           
        f.value = step
        f.description = f"Ep{epoch}:{step}/{max_steps}"
        if (step) % eval_interval == 0:
            
            t1 = time.time()
            print(f"Time left: {datetime.timedelta(seconds=int((t1-t0)*(max_steps-step)))}s")
            
            print('Step: %d' % step)
            test_pred = model.classify(test_images).argmax(-1)
            conf_mtx = semisup.confusion_matrix(test_labels, test_pred, NUM_LABELS)
            test_err = (test_labels != test_pred).mean() * 100
            print(conf_mtx)
            print('Test error: %.2f %%' % test_err)
            print(f'Loss: {loss}')
            print()
    
            test_summary = tf.Summary(
                value=[tf.Summary.Value(
                    tag='Test Err', simple_value=test_err)])
    
            summary_writer.add_summary(summaries, step)
            summary_writer.add_summary(test_summary, step)

            print(f"saving model - step {step}")
            saver.save(sess, store_dir, global_step=model.step)

IntProgress(value=0, layout=Layout(width='100%'), max=1002)

INFO:tensorflow:Restoring parameters from logs/training/mnist-mnistmK/model-501
Time left: 2:24:15s
Step: 600
[[113   0  16  11   0   1  10   1  20   3]
 [  6 116  23  12  14   8   8   8  39   0]
 [  4   1 174  18   1   0   1   5  14   1]
 [  6   4  20 127   0  15   1   5  26   3]
 [  1   4  14  14 125   4   8  11  27   9]
 [  2   0   2  20   2 116   7   2  27   1]
 [  3   2   8  14   5  15 111   0  19   1]
 [  1   2  21  13   9   3   0 132  19   5]
 [  4   1  14  20   1   8   6   6 130   2]
 [  8   0   9  31  11  14   2  17  23  79]]
Test error: 38.85 %
Loss: 82.96365356445312

saving model - step 600
Time left: 1:54:11s
Step: 700
[[117   1  14  12   0   1   6   2  19   3]
 [  6 127  17  12  12   9   4  14  33   0]
 [  5   1 171  19   2   0   0   5  14   2]
 [  6   4  16 131   0  13   1   6  27   3]
 [  1   5  12  18 123   5   8  14  23   8]
 [  2   0   1  22   2 116   6   2  27   1]
 [  6   3   7  15   7  12 109   1  17   1]
 [  1   3  19  15   8   3   0 135  19   2]
 [  6   2  13  2

In [9]:
all_steps = list(range(1,1002,100))

p_aba_list = []
match_ab_list = []
p_ab_list = []
semb_list = []
uemb_list = []
slabel_list = []
ulabel_list = []

for train_step in all_steps:
    if train_step <= 501:
        model_checkpoint = f"{log_dir}-{train_step}"
    else:
        model_checkpoint = f"{store_dir}-{train_step}"

    with tf.Session(graph=graph) as sess:
        
        saver = tf.train.Saver()
        saver.restore(sess, model_checkpoint)
        
        sess.run(unsup_it.initializer)
        
        pab, slabel, ulabel, ab, aba, semb, uemb = sess.run([p_ab, t_sup_labels, t_unsup_labels, match_ab, p_aba, t_sup_emb, t_unsup_emb])
        p_aba_list.append(aba)
        match_ab_list.append(ab)
        p_ab_list.append(pab)
        semb_list.append(semb)
        uemb_list.append(uemb)
        slabel_list.append(slabel)
        ulabel_list.append(ulabel)

INFO:tensorflow:Restoring parameters from logs/training/mnist-mnistmK/model-1
INFO:tensorflow:Restoring parameters from logs/training/mnist-mnistmK/model-101
INFO:tensorflow:Restoring parameters from logs/training/mnist-mnistmK/model-201
INFO:tensorflow:Restoring parameters from logs/training/mnist-mnistmK/model-301
INFO:tensorflow:Restoring parameters from logs/training/mnist-mnistmK/model-401
INFO:tensorflow:Restoring parameters from logs/training/mnist-mnistmK/model-501
INFO:tensorflow:Restoring parameters from logs/training/mnist-mnistK-s0.5/model-601
INFO:tensorflow:Restoring parameters from logs/training/mnist-mnistK-s0.5/model-701
INFO:tensorflow:Restoring parameters from logs/training/mnist-mnistK-s0.5/model-801
INFO:tensorflow:Restoring parameters from logs/training/mnist-mnistK-s0.5/model-901
INFO:tensorflow:Restoring parameters from logs/training/mnist-mnistK-s0.5/model-1001


In [20]:
with tf.Session(graph=graph) as sess:
    saver = tf.train.Saver()
    
    model_checkpoint = f"{store_dir}-{601}"
    saver.restore(sess, model_checkpoint)
    sess.run(unsup_it.initializer)
    
    laba, mabgr, pabgr, pabagr, sgr, ugr = sess.run([loss_aba, mab_dt, pab_dt, paba_dt, semb_dt, uemb_dt])

INFO:tensorflow:Restoring parameters from logs/training/mnist-mnistK-s40/model-601


In [27]:
pabagr

array([[-0.00528175, -0.00561872, -0.00520749, ...,  0.00059999,
         0.00059999,  0.00059999],
       [-0.00529621, -0.00560133, -0.00522016, ...,  0.00059999,
         0.00059999,  0.00059999],
       [-0.00530337, -0.00563986, -0.00519285, ...,  0.00059999,
         0.00059999,  0.00059999],
       ...,
       [ 0.00059999,  0.00059999,  0.00059999, ..., -0.00523213,
        -0.00545448, -0.00540969],
       [ 0.00059999,  0.00059999,  0.00059999, ..., -0.00523463,
        -0.00544493, -0.00540712],
       [ 0.00059999,  0.00059999,  0.00059999, ..., -0.00528052,
        -0.00549948, -0.00536054]], dtype=float32)

In [28]:
pabgr

array([[ 3.31671072e-05,  2.22829549e-05,  3.88521403e-06, ...,
         3.97997937e-05,  2.60515644e-05, -1.02260929e-05],
       [ 3.20304352e-05,  2.11506140e-05,  2.69929842e-06, ...,
         3.86981919e-05,  2.49356308e-05, -1.14620943e-05],
       [ 3.28725364e-05,  2.18775967e-05,  3.56622877e-06, ...,
         3.94613635e-05,  2.56402636e-05, -1.04909595e-05],
       ...,
       [-8.02024297e-05,  2.86057038e-05, -3.66997119e-05, ...,
        -1.15878793e-05, -5.89669944e-05,  2.49999139e-05],
       [-7.96724198e-05,  2.90179014e-05, -3.62238206e-05, ...,
        -1.11469562e-05, -5.85033558e-05,  2.54311744e-05],
       [-8.08653858e-05,  2.87733856e-05, -3.68884212e-05, ...,
        -1.14469440e-05, -5.83730289e-05,  2.45909032e-05]], dtype=float32)

In [29]:
mabgr

array([[ 6.51997567e-08,  5.30075681e-08,  1.75683752e-08, ...,
         8.28770226e-08,  5.79718034e-08, -1.34038212e-08],
       [ 3.70654334e-08,  2.02222559e-08, -1.61795572e-08, ...,
         5.50345831e-08,  3.00880885e-08, -4.82694666e-08],
       [ 7.75299043e-08,  5.55734836e-08,  2.91538385e-08, ...,
         9.63606581e-08,  6.20624547e-08, -3.77756582e-09],
       ...,
       [-1.53317416e-07,  6.22706153e-08, -6.39316724e-08, ...,
        -1.10663221e-08, -9.49409724e-08,  6.75445548e-08],
       [-1.96473820e-07,  4.34731184e-08, -8.87426950e-08, ...,
        -3.11478203e-08, -1.15436286e-07,  4.21994919e-08],
       [-1.54327836e-07,  5.69168535e-08, -7.82367806e-08, ...,
        -2.56784300e-08, -1.49675856e-07,  4.48925306e-08]], dtype=float32)

In [34]:
sgr

array([[-1.73160100e-08,  1.43034535e-08,  9.16143694e-09, ...,
        -1.38655025e-08, -7.06721293e-10, -3.01570182e-08],
       [ 3.01983434e-08,  2.41971261e-08, -1.71434849e-08, ...,
        -5.27470900e-10,  6.03642292e-09,  4.04439788e-08],
       [-1.86297182e-08,  3.54174041e-08,  1.08918679e-08, ...,
        -1.76803141e-08, -1.73815522e-08, -2.46733158e-08],
       ...,
       [ 3.09021608e-08, -4.55078919e-10,  1.29072335e-08, ...,
        -1.28435185e-09,  2.12495110e-09,  2.38004425e-08],
       [ 1.80136139e-09, -1.01491429e-08,  5.80294923e-09, ...,
         1.83752409e-08,  2.34520225e-09,  1.82092719e-09],
       [ 2.03308250e-08, -4.96763608e-09,  1.00391846e-08, ...,
        -2.35994890e-09,  4.96717723e-10,  1.83357010e-08]], dtype=float32)

In [10]:
sorted_uemb_list     = []
sorted_match_ab_list = []

from scipy.special import softmax

for idx in range(len(uemb_list)):
    sorted_uemb_list.append(uemb_list[idx][ulabel_list[idx].argsort()])
    sorted_match_ab_list.append(match_ab_list[idx][:,ulabel_list[idx].argsort()])

In [12]:

from ipywidgets import IntSlider, RadioButtons
from IPython.html.widgets import *
import matplotlib.pyplot as plt

def plot_img(array, step):
    if array == 0:
        a = p_aba_list
    elif array == 1:
        a = match_ab_list
    elif array == 2:
        a = sorted_match_ab_list
    else:
        a = p_ab_list 
            
            
    fig = plt.figure()
    plt.imshow(a[(step-1)//100])
    plt.colorbar()

interact(
    lambda step, array: plot_img(array, step),
    step = IntSlider(
        value=901,
        min=1,
        max=1501,
        step=100,
        description='Train Step',
        continuous_update=True,
        orientation='horizontal',
    ),
    array = RadioButtons(
        options=[('p_aba', 0), ("match_ab", 1), ("sorted_match_ab", 2), ("p_ab", 3)],
    )
)

interactive(children=(IntSlider(value=901, description='Train Step', max=1501, min=1, step=100), RadioButtons(…

<function __main__.<lambda>(step, array)>

In [ ]:
idx = 6

my_semb = semb_list[idx]
my_uemb = uemb_list[idx]

embedding_dim = my_semb.shape[1]

reshaped_semb = my_semb.reshape((-1, 1, embedding_dim))
reshaped_uemb = my_uemb.reshape((-1, 1, embedding_dim))

repeat_semb = np.repeat(reshaped_semb, unsup_batch_size, axis=1)
repeat_uemb = np.repeat(reshaped_uemb, sup_per_batch*NUM_LABELS, axis=1)

uemb_T = np.transpose(repeat_uemb, axes=[1,0,2])

sigma = 40
my_match_ab = np.exp(-(np.linalg.norm(repeat_semb - uemb_T, axis=2)**2)/(2*sigma**2))


from sklearn.utils.extmath import softmax
plt.imshow(softmax(my_match_ab))

In [35]:
g2 = tf.Graph()

with g2.as_default():
    
    x = tf.ones([1])
    t = tf.gradients([x**2], [x])


with tf.Session(graph = g2) as sesh:
    print(sesh.run(t))

[array([2.], dtype=float32)]
